## 데이터 분석


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive



### Attribute 변환
id -> id  
hour -> 시간  
temperature -> 기온  
precipitation -> 비  
windspeed -> 풍속    
humidity -> 습도  
visibility -> 가시성  
ozone -> 오존  
pm10 -> 미세먼지10  
pm2.5 -> 미세먼지2.5    
count -> count   

In [3]:
data_folder = '/content/gdrive/MyDrive/data/BikeRental_Data/sample_data/'
train = pd.read_csv(data_folder+'train.csv')
test = pd.read_csv(data_folder+'test.csv')
train.columns = ['id', '시간', '기온', '비',
       '풍속', '습도', '가시성',
       '오존', '미세먼지10', '미세먼지2.5', 'count']
test.columns = ['id', '시간', '기온', '비',
       '풍속', '습도', '가시성',
       '오존', '미세먼지10', '미세먼지2.5']

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       1459 non-null   int64  
 1   시간       1459 non-null   int64  
 2   기온       1457 non-null   float64
 3   비        1457 non-null   float64
 4   풍속       1450 non-null   float64
 5   습도       1457 non-null   float64
 6   가시성      1457 non-null   float64
 7   오존       1383 non-null   float64
 8   미세먼지10   1369 non-null   float64
 9   미세먼지2.5  1342 non-null   float64
 10  count    1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [5]:
train.head()

,id,시간,기온,비,풍속,습도,가시성,오존,미세먼지10,미세먼지2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


## 결측치 확인 및 처리

In [6]:
train_null = train.isnull().sum(axis=1)
train_null.value_counts()

0    1328
3      66
1      42
2      20
8       2
4       1
dtype: int64

In [7]:
test_null = test.isnull().sum(axis=1)
test_null.value_counts()

0    674
3     30
2      5
1      5
8      1
dtype: int64

각 row에 대해서 결측치가 절반 이상 있는 데이터는 거의 없다.  
train의 경우 기온이 결측치인 경우, 해당 데이터들은 모두 8개의 결측치를 갖는다.  

In [8]:
train.isnull().sum()

id           0
시간           0
기온           2
비            2
풍속           9
습도           2
가시성          2
오존          76
미세먼지10      90
미세먼지2.5    117
count        0
dtype: int64

In [9]:
test.isnull().sum()

id          0
시간          0
기온          1
비           1
풍속          1
습도          1
가시성         1
오존         35
미세먼지10     37
미세먼지2.5    36
dtype: int64

### 결측치 정리 - train

  count에 대한 결측치가 없으므로 무조건 삭제할 데이터는 없다.  
  2.5 미세먼지가 결측치가 많으나, 컬럼을 살제할지 채울지 여부는 데이터 특성을 보고 결정한다.   
  기상청 날씨와 연동하여 채워넣기도 가능하다.  

  온도의 경우 비가 왔는지 여부를 확인하여 앞 뒤 날짜의 평균으로 채운다.

## 기온 분석
비 여부와 전날, 다음날 온도를 보고 어떤 값으로 채울지 여부를 정해야할 수 있다.

In [10]:
train_t_na = train[train['기온'].isna()]
test_t_na = test[test['기온'].isna()]
print(train_t_na)
print(test_t_na)


        id  시간  기온   비  풍속  습도  가시성  오존  미세먼지10  미세먼지2.5  count
934   1420   0 NaN NaN NaN NaN  NaN NaN     NaN      NaN   39.0
1035  1553  18 NaN NaN NaN NaN  NaN NaN     NaN      NaN    1.0
       id  시간  기온   비  풍속  습도  가시성  오존  미세먼지10  미세먼지2.5
653  1943  19 NaN NaN NaN NaN  NaN NaN     NaN      NaN


train 934, 1035와 test 653은 결측치가 심하다.  
데이터를 확인한 결과, index 별 날짜 연관성이 적어보이므로 앞 뒤 데이터의 평균으로 채운다는 방법은 기각, 데이터를 삭제한다.  
test 결측치는 평균값으로 대체한다.


In [11]:
train.iloc[930:940]

,id,시간,기온,비,풍속,습도,가시성,오존,미세먼지10,미세먼지2.5,count
930,1414,0,18.4,0.0,2.3,44.0,2000.0,0.037,26.0,21.0,128.0
931,1415,20,10.7,0.0,3.8,38.0,1246.0,0.064,72.0,60.0,87.0
932,1417,4,4.2,0.0,1.1,77.0,1258.0,0.024,24.0,NaN,16.0
933,1419,4,11.0,0.0,0.9,84.0,532.0,0.052,60.0,33.0,22.0
934,1420,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0
935,1421,10,11.2,0.0,1.5,58.0,1275.0,0.019,66.0,24.0,70.0
936,1422,3,15.2,0.0,0.7,44.0,2000.0,0.041,20.0,17.0,35.0
937,1426,12,9.9,0.0,2.0,57.0,1933.0,0.030,19.0,NaN,78.0
938,1427,14,25.0,0.0,3.1,44.0,1989.0,0.053,27.0,17.0,161.0
939,1428,9,11.8,0.0,2.0,80.0,584.0,0.017,43.0,33.0,113.0


In [12]:
### train 기온 결측치 삭제
train = train.drop([934,1035],axis=0)


In [13]:
train.iloc[1030:1040]

,id,시간,기온,비,풍속,습도,가시성,오존,미세먼지10,미세먼지2.5,count
1031,1547,17,19.8,0.0,2.9,68.0,636.0,0.050,63.0,39.0,138.0
1032,1548,19,25.1,0.0,4.7,55.0,1222.0,0.083,60.0,38.0,293.0
1033,1549,6,15.6,0.0,1.1,63.0,1211.0,0.023,45.0,31.0,30.0
1034,1551,0,19.7,0.0,3.4,76.0,749.0,0.065,48.0,31.0,100.0
1036,1557,23,9.5,0.0,2.9,37.0,2000.0,0.036,56.0,15.0,58.0
1037,1558,7,7.0,0.0,1.1,75.0,1981.0,0.023,57.0,14.0,88.0
1038,1560,5,12.8,0.0,0.3,74.0,1333.0,0.006,36.0,20.0,14.0
1039,1561,6,14.0,0.0,0.7,51.0,2000.0,0.013,42.0,26.0,33.0
1040,1562,14,14.1,1.0,3.2,50.0,1645.0,0.045,46.0,39.0,2.0
1041,1564,13,21.2,0.0,4.2,26.0,1542.0,0.056,129.0,27.0,108.0


In [14]:

### test 기온 결측치 채우기
sample = test[test['시간']==19].mean()
print(test.iloc[653])
sample[['id','시간']] = test.iloc[653][['id','시간']]
print(sample)
test.iloc[653]= sample
# test.iloc[653]

id         1943.0
시간           19.0
기온            NaN
비             NaN
풍속            NaN
습도            NaN
가시성           NaN
오존            NaN
미세먼지10        NaN
미세먼지2.5       NaN
Name: 653, dtype: float64
id         1943.000000
시간           19.000000
기온           26.110345
비             0.068966
풍속            3.541379
습도           47.689655
가시성        1561.758621
오존            0.054207
미세먼지10       39.214286
미세먼지2.5      26.071429
dtype: float64


In [16]:
print(train.isnull().sum())
print(test.isnull().sum())

id           0
시간           0
기온           0
비            0
풍속           7
습도           0
가시성          0
오존          74
미세먼지10      88
미세먼지2.5    115
count        0
dtype: int64
id          0
시간          0
기온          0
비           0
풍속          0
습도          0
가시성         0
오존         34
미세먼지10     36
미세먼지2.5    35
dtype: int64


## 미세먼지, 오존 분석
test 데이터셋의 경우, 결측치가 있는 데이터가 총 40개이다.  
그런데 오존, 미세먼지의 결측치 데이터가 35, 37, 36이므로 세 개의 데이터는 함께 묶여서 결측치일 확률이 높다.


In [17]:
all_na = test['오존'].isna() & test['미세먼지10'].isna() & test['미세먼지2.5'].isna()
all_na.sum()

30

확인 결과 30개의 데이터에서 3개의 특성이 결측치인 것을 확인할 수 있다.  
해당 데이터는 다른 특성과의 상관관계 비교를 통해 해당 특성값에 따른 회귀분석 모델로 채우는 시도를 해본다.  

In [18]:
train.corr()

,id,시간,기온,비,풍속,습도,가시성,오존,미세먼지10,미세먼지2.5,count
id,1.000000,-0.010807,-0.000029,-0.056267,-0.003353,-0.017948,0.008950,0.055234,-0.025510,0.003545,-0.001212
시간,-0.010807,1.000000,0.407306,0.021646,0.462797,-0.329612,0.176491,0.390188,-0.035907,-0.061229,0.627227
기온,-0.000029,0.407306,1.000000,-0.097056,0.375618,-0.496088,0.197417,0.541228,-0.003830,-0.078665,0.619404
비,-0.056267,0.021646,-0.097056,1.000000,0.022746,0.276481,-0.217155,-0.062461,-0.051266,0.004742,-0.163985
풍속,-0.003353,0.462797,0.375618,0.022746,1.000000,-0.433012,0.252092,0.520526,0.010176,-0.199113,0.459906
습도,-0.017948,-0.329612,-0.496088,0.276481,-0.433012,1.000000,-0.592244,-0.421047,-0.108106,0.167736,-0.471142
가시성,0.008950,0.176491,0.197417,-0.217155,0.252092,-0.592244,1.000000,0.101899,-0.403277,-0.644989,0.299094
오존,0.055234,0.390188,0.541228,-0.062461,0.520526,-0.421047,0.101899,1.000000,0.113015,0.017313,0.477614
미세먼지10,-0.025510,-0.035907,-0.003830,-0.051266,0.010176,-0.108106,-0.403277,0.113015,1.000000,0.489558,-0.114288
미세먼지2.5,0.003545,-0.061229,-0.078665,0.004742,-0.199113,0.167736,-0.644989,0.017313,0.489558,1.000000,-0.134293


In [19]:
test.corr()

,id,시간,기온,비,풍속,습도,가시성,오존,미세먼지10,미세먼지2.5
id,1.000000,-0.023110,-0.022844,-0.079947,0.021403,0.024850,-0.021348,0.011208,0.022812,0.037106
시간,-0.023110,1.000000,0.482520,0.006409,0.503893,-0.355514,0.203318,0.370635,0.105205,0.060836
기온,-0.022844,0.482520,1.000000,-0.233590,0.372995,-0.579636,0.143382,0.552937,0.344295,0.312590
비,-0.079947,0.006409,-0.233590,1.000000,-0.098479,0.390698,-0.312082,-0.079497,-0.152253,-0.071358
풍속,0.021403,0.503893,0.372995,-0.098479,1.000000,-0.396323,0.212516,0.522121,0.138947,0.048056
습도,0.024850,-0.355514,-0.579636,0.390698,-0.396323,1.000000,-0.689915,-0.404531,-0.008559,0.198363
가시성,-0.021348,0.203318,0.143382,-0.312082,0.212516,-0.689915,1.000000,-0.030653,-0.446526,-0.607882
오존,0.011208,0.370635,0.552937,-0.079497,0.522121,-0.404531,-0.030653,1.000000,0.443593,0.397584
미세먼지10,0.022812,0.105205,0.344295,-0.152253,0.138947,-0.008559,-0.446526,0.443593,1.000000,0.909827
미세먼지2.5,0.037106,0.060836,0.312590,-0.071358,0.048056,0.198363,-0.607882,0.397584,0.909827,1.000000


오존은 기온과, 미세먼지는 가시성과 비교적 큰 상관관계를 갖는 것을 알 수 있다.  
해당 값들은 회귀분석을 통해 결측치를 채운다.  

In [20]:
from sklearn import linear_model

In [21]:
data = train[['기온','오존']].dropna(axis=0)
x = data[['기온']]
y = data['오존']
lin_model_ozon = linear_model.LinearRegression()
lin_fit_ozon = lin_model_ozon.fit(x,y)


In [22]:
data = train[['가시성','미세먼지10','미세먼지2.5']].dropna(axis=0)
x = data[['가시성']]
y = data['미세먼지10']
lin_model_10 = linear_model.LinearRegression()
lin_fit_10 = lin_model_10.fit(x,y)

y = data['미세먼지10']
lin_model_25 = linear_model.LinearRegression()
lin_fit_25 = lin_model_25.fit(x,y)


In [23]:
### 회귀 분석이 불가능한 케이스는 없으므로 결측치를 채운다.
print((train['오존'].isna() & train['기온'].isna()).sum())
print((train['가시성'].isna() & train['미세먼지10'].isna()& train['미세먼지2.5'].isna()).sum())
print((test['오존'].isna() & test['기온'].isna()).sum())
print((test['가시성'].isna() & test['미세먼지10'].isna()& test['미세먼지2.5'].isna()).sum())

0
0
0
0


In [24]:
na_idx = train['오존'].isna()
na_val = lin_model_ozon.predict(train.loc[na_idx][['기온']])
train['오존'][na_idx] = na_val
na_idx = test['오존'].isna()
na_val = lin_model_ozon.predict(test.loc[na_idx][['기온']])
test['오존'][na_idx] = na_val

na_idx = train['미세먼지10'].isna()
na_val = lin_model_10.predict(train.loc[na_idx][['가시성']])
na_idx = train['미세먼지2.5'].isna()
na_val = lin_model_25.predict(train.loc[na_idx][['가시성']])
train['미세먼지10'][na_idx] = na_val
train['미세먼지2.5'][na_idx] = na_val

na_idx = test['미세먼지10'].isna()
na_val = lin_model_10.predict(test.loc[na_idx][['가시성']])
na_idx = test['미세먼지2.5'].isna()
na_val = lin_model_25.predict(test.loc[na_idx][['가시성']])
test['미세먼지10'][na_idx] = na_val
test['미세먼지2.5'][na_idx] = na_val

<ipython-input-24-995ab28f458c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['오존'][na_idx] = na_val
<ipython-input-24-995ab28f458c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['오존'][na_idx] = na_val
<ipython-input-24-995ab28f458c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['미세먼지10'][na_idx] = na_val
<ipython-input-24-995ab28f458c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [26]:
print(train.isnull().sum())
print(test.isnull().sum())

id         0
시간         0
기온         0
비          0
풍속         7
습도         0
가시성        0
오존         0
미세먼지10     1
미세먼지2.5    0
count      0
dtype: int64
id         0
시간         0
기온         0
비          0
풍속         0
습도         0
가시성        0
오존         0
미세먼지10     1
미세먼지2.5    0
dtype: int64


결측치가 많이 해결된 것을 볼 수 있다.
나머지 결측치는 0으로 대체한다

```
# 코드로 형식 지정됨
```



In [27]:
train.fillna(0,inplace = True)
test.fillna(0,inplace=True)

## 모델 정의 및 학습

In [28]:
train_x = train.drop(['count'],axis = 1)
train_y = train['count']

In [29]:
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)

RandomForestRegressor()

## 학습 된 모델로 예측 데이터 생성

In [30]:
pred = model.predict(test)

## 제출파일 생성

In [31]:
submission = pd.read_csv(data_folder+'submission.csv')
submission

,id,count
0,0,NaN
1,1,NaN
2,2,NaN
3,4,NaN
4,5,NaN
...,...,...
710,2148,NaN
711,2149,NaN
712,2165,NaN
713,2166,NaN


In [32]:
submission['count'] = pred

In [33]:
submission

,id,count
0,0,101.68
1,1,237.13
2,2,85.97
3,4,30.08
4,5,73.60
...,...,...
710,2148,53.57
711,2149,63.71
712,2165,129.00
713,2166,161.14


In [34]:
submission.to_csv('/content/gdrive/MyDrive/data/BikeRental_Data/prediction2.csv',index = False)

In [36]:
train.to_csv('/content/gdrive/MyDrive/data/BikeRental_Data/train2.csv')
test.to_csv('/content/gdrive/MyDrive/data/BikeRental_Data/test2.csv')

## 결과

45.7239089422 -> 45.6679876769